<a href="https://colab.research.google.com/github/iparinile/ML_learning/blob/master/Copy_of_dogs_vs_cats_cnn(%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BA%D0%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознавание собак и кошек на изображениях с помощью сверточной нейронной сети

**Источник данных** - соревнования Kaggle [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/data).

Для распознавания используется сверточная нейронная сеть.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования. Можно использовать пример в ноутбуке dogs_vs_cats_data_preparation

In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import keras
import zipfile

Using TensorFlow backend.


In [2]:
# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Распаковка архива с подготовленным датасетом
with zipfile.ZipFile('/content/drive/My Drive/dogs-vs-cats-prepared-dataset.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/')

In [0]:
# Каталог с данными для обучения
train_dir = 'ds/train'
# Каталог с данными для проверки
val_dir = 'ds/val'
# Каталог с данными для тестирования
test_dir = 'ds/test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 32
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

## Создаем сверточную нейронную сеть

**Архитектура сети**
1. Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
2. Слой подвыборки, выбор максимального значения из квадрата 2х2
3. Слой свертки, размер ядра 3х3, количество карт признаков - 32 шт., функция активации ReLU.
4. Слой подвыборки, выбор максимального значения из квадрата 2х2
5. Слой свертки, размер ядра 3х3, количество карт признаков - 64 шт., функция активации ReLU.
6. Слой подвыборки, выбор максимального значения из квадрата 2х2
7. Слой преобразования из двумерного в одномерное представление
8. Полносвязный слой, 64 нейрона, функция активации ReLU.
9. Слой Dropout.
10. Выходной слой, 1 нейрон, функция активации sigmoid

Слои с 1 по 6 используются для выделения важных признаков в изображении, а слои с 7 по 10 - для классификации.

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Компилируем нейронную сеть

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Создаем генератор изображений

Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [0]:
datagen = ImageDataGenerator(rotation_range=15,
                            rescale=1./255,
                            shear_range=0.1,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            width_shift_range=0.1,
                            height_shift_range=0.1)


Генератор данных для обучения на основе изображений из каталога

In [8]:
%cd /content
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

/content
Found 17500 images belonging to 2 classes.


Генератор данных для проверки на основе изображений из каталога

In [9]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 3750 images belonging to 2 classes.


Генератор данных для тестирования на основе изображений из каталога

In [10]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


# Обучаем модель с использованием генераторов

train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [0]:
es = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=0, mode='auto')

In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[es],
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
546/546 [==============================] - 175s 321ms/step - loss: 0.6722 - acc: 0.5749 - val_loss: 0.6012 - val_acc: 0.6704
Epoch 2/30
546/546 [==============================] - 175s 320ms/step - loss: 0.6058 - acc: 0.6729 - val_loss: 0.5638 - val_acc: 0.7150
Epoch 3/30
546/546 [==============================] - 175s 320ms/step - loss: 0.5606 - acc: 0.7068 - val_loss: 0.5081 - val_acc: 0.7439
Epoch 4/30
546/546 [==============================] - 175s 320ms/step - loss: 0.5261 - acc: 0.7443 - val_loss: 0.4687 - val_acc: 0.7807
Epoch 5/30
546/546 [==============================] - 175s 320ms/step - loss: 0.5040 - acc: 0.7513 - val_loss: 0.4688 - val_acc: 0.7772
Epoch 6/30
546/546 [==============================] - 174s 318ms/step - loss: 0.4781 - acc: 0.7735 - val_loss: 0.4413 - val_acc: 0.7962
Epoch 7/30
546/546 [==============================] - 173s 317ms/step - loss: 0.4620 - acc: 0.7819 - val_loss: 0.4147 - val_acc: 0.8069
Epoch 8/30
546/546 [============================

# Оцениваем качество работы сети с помощью генератора

In [14]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 88.06%
